
# DGLを用いたノード分類

GNNはグラフ上の多くの機械学習タスクに対して強力なツールである。今回のチュートリアルでは、ノード分類のためにGNNを使用する基本的なワークフローを学ぶ。
つまり、グラフ内のノードのカテゴリを予測する。

### 目標

-  DGL提供のデータセットを読み込む。
-  DGL提供のnnモジュールを使用してGNNモデルを構築する。
-  ノード分類のためのGNNモデルを学習し、評価する。

このチュートリアルは、PyTorchを使用してニューラルネットワークを構築する経験があることを前提としている。

1. Python=3.11.8
2. Pytorch==2.1.2+cu121
3. dgl==2.1.0.cu121
4. dglgo==0.0.2

In [4]:
# ライブラリのインポート
import os

os.environ["DGLBACKEND"] = "pytorch" # バックエンドとしてPytorch, Tensorflow, MXNetを指定可能。今回はPytorch
import dgl
#import dgl.data 
import torch
import torch.nn as nn
import torch.nn.functional as F

## GNNを用いたノード分類の概要
グラフデータに対する最も一般的かつ広く用いられているタスクの1つに、ノード分類がある。ノード分類においては、モデルは各ノードのカテゴリを予測する。グラフニューラルネットワーク以前は、多くの手法が接続性 (connectivity) のみ（DeepWalkやnode2vecなど）、あるいは接続性とノード自身の特徴の単純な組み合わせを使用していた。それに対して、GNNは局所的な近傍の接続性と特徴を組み合わせてノード表現を得ることが可能である。

[Kipfら](https://arxiv.org/abs/1609.02907)は、ノード分類問題を半教師あり学習によるノード分類タスクとして定式化した例である。ラベル付けされたノードのごく一部だけを使って、グラフニューラルネットワーク（GNN）は他のノードのカテゴリを正確に予測できる。このチュートリアルでは、引用ネットワークであるCoraデータセットを用いて、論文をノードとし、引用をエッジとして、ごく少数のラベルで半教師ありノード分類のためのGNNを構築する方法を示す。タスクは、与えられた論文のカテゴリを予測することだ。各論文ノードは、[論文](https://arxiv.org/abs/1609.02907)のセクション5.2で説明されているように、単語カウントベクトルを特徴として含んでおり、合計が1になるように正規化されている。

## Coraデータセットの読み込み


Coraデータセットは、7つのクラスのいずれかに分類された2708の科学的出版物で構成されている。引用ネットワークは5429のリンクで構成されています。データセット内の各出版物は、辞書の対応する単語の有無を示す0/1値の単語ベクトルによって記述されます。辞書は1433のユニークな単語で構成されています。

In [6]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


このチュートリアルで使用されているCoraデータセットは、1つのグラフのみで構成されている。DGLのDatasetオブジェクトは、1つまたは複数のグラフを含むことができる。

In [10]:
g = dataset[0]

In [30]:
edges = torch.cat([g.edges()[0].reshape(1, -1), g.edges()[1].reshape(1, -1)], dim=0)
edges.shape

torch.Size([2, 10556])

In [31]:
edges

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

DGLグラフは、```ndata```と```edata```と呼ばれる2つの辞書のような属性にノードの特徴とエッジの特徴を格納できる。DGLのCoraデータセットでは、グラフには以下のようなノードの特徴が含まれている：

- ``train_mask``: trainingセットにノードが含まれているかどうかを示すブール値のテンソル。

- ``val_mask``: validationセットにノードが含まれているかどうかを示すブール値のテンソル。

- ``test_mask``: testセットにノードが含まれているかどうかを示すブール値のテンソル。

- ``label``: 真のノードのラベル (論文のカテゴリ)。

-  ``feat``: ノードの特徴量 (単語カウントベクトル)。




In [4]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


In [5]:
# 2708はノード数 (論文数)、1433は特徴量の次元数 (単語数)
print("feat:", g.ndata["feat"].shape)
print("label:", g.ndata["label"].shape)
print("train_mask:", g.ndata["train_mask"].sum())
print("val_mask:", g.ndata["val_mask"].sum())
print("test_mask:", g.ndata["test_mask"].sum())

feat: torch.Size([2708, 1433])
label: torch.Size([2708])
train_mask: tensor(140)
val_mask: tensor(500)
test_mask: tensor(1000)


エッジの特徴は、このチュートリアルでは使用されていない。

## Graph Convolutional Network (GCN) の構築

このチュートリアルでは、2層の[Graph Convolutional Network (GCN)](http://tkipf.github.io/graph-convolutional-networks/)
を構築する。各層は、隣接情報を集約して新しいノード表現を計算する。

多層のGCNを構築するには、``dgl.nn.GraphConv``モジュールを積み重ねるだけでよい。これは、``torch.nn.Module``を継承している。


In [33]:
from dgl.nn import GraphConv

# GCNモデルの定義
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# 入力特徴量の次元数、中間層の次元数、出力次元数を指定してモデルのインスタンスを生成。今回は (1433, 16, 7)
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

DGLは、多くの一般的な隣接情報集約モジュールの実装を提供している。1行のコードで簡単に呼び出すことができる。

## GCNの学習

GCNの学習は、他のPyTorchニューラルネットワークの学習と同様である。

In [35]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]

    
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )

model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.142 (best 0.142), test acc: 0.150 (best 0.150)
In epoch 5, loss: 1.894, val acc: 0.340 (best 0.358), test acc: 0.359 (best 0.360)
In epoch 10, loss: 1.816, val acc: 0.428 (best 0.428), test acc: 0.467 (best 0.467)
In epoch 15, loss: 1.713, val acc: 0.514 (best 0.514), test acc: 0.541 (best 0.541)
In epoch 20, loss: 1.587, val acc: 0.560 (best 0.560), test acc: 0.587 (best 0.587)
In epoch 25, loss: 1.440, val acc: 0.642 (best 0.642), test acc: 0.636 (best 0.636)
In epoch 30, loss: 1.278, val acc: 0.660 (best 0.660), test acc: 0.670 (best 0.670)
In epoch 35, loss: 1.106, val acc: 0.674 (best 0.676), test acc: 0.704 (best 0.698)
In epoch 40, loss: 0.934, val acc: 0.716 (best 0.716), test acc: 0.728 (best 0.728)
In epoch 45, loss: 0.770, val acc: 0.732 (best 0.732), test acc: 0.732 (best 0.732)
In epoch 50, loss: 0.624, val acc: 0.744 (best 0.744), test acc: 0.746 (best 0.741)
In epoch 55, loss: 0.499, val acc: 0.752 (best 0.752), test acc: 0.751 (best 0

## GPUでの学習
GPUでの学習には、Pytorchと同様モデルとグラフの両方を``to``メソッドを使ってGPUに配置する必要がある。

```python

   g = g.to('cuda')
   model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
   train(g, model)



In [36]:
# GPUを使って学習
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.947, val acc: 0.102 (best 0.102), test acc: 0.114 (best 0.114)
In epoch 5, loss: 1.903, val acc: 0.336 (best 0.406), test acc: 0.335 (best 0.433)
In epoch 10, loss: 1.826, val acc: 0.402 (best 0.406), test acc: 0.429 (best 0.433)
In epoch 15, loss: 1.729, val acc: 0.540 (best 0.540), test acc: 0.575 (best 0.575)
In epoch 20, loss: 1.607, val acc: 0.612 (best 0.612), test acc: 0.626 (best 0.626)
In epoch 25, loss: 1.466, val acc: 0.642 (best 0.642), test acc: 0.658 (best 0.658)
In epoch 30, loss: 1.307, val acc: 0.676 (best 0.676), test acc: 0.690 (best 0.690)
In epoch 35, loss: 1.138, val acc: 0.700 (best 0.700), test acc: 0.708 (best 0.708)
In epoch 40, loss: 0.968, val acc: 0.722 (best 0.722), test acc: 0.727 (best 0.727)
In epoch 45, loss: 0.806, val acc: 0.726 (best 0.728), test acc: 0.743 (best 0.734)
In epoch 50, loss: 0.661, val acc: 0.746 (best 0.746), test acc: 0.747 (best 0.747)
In epoch 55, loss: 0.535, val acc: 0.748 (best 0.750), test acc: 0.748 (best 0

今回はGPUよりもCPUのほう速かった。おそらく学習が短すぎてテンソルとモデルの転送にかかる時間が大きかったためだと思われる。

## What’s next?

-  :doc:`How does DGL represent a graph <2_dglgraph>`?
-  :doc:`Write your own GNN module <3_message_passing>`.
-  :doc:`Link prediction (predicting existence of edges) on full
   graph <4_link_predict>`.
-  :doc:`Graph classification <5_graph_classification>`.
-  :doc:`Make your own dataset <6_load_data>`.
-  `The list of supported graph convolution
   modules <apinn-pytorch>`.
-  `The list of datasets provided by DGL <apidata>`.




In [7]:
# Thumbnail credits: Stanford CS224W Notes
# sphinx_gallery_thumbnail_path = '_static/blitz_1_introduction.png'